<a href="https://colab.research.google.com/github/pahrul23/pahrul-skripsi/blob/main/TES_AJAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('drive/MyDrive/Wira/labelV1.csv')
df.dropna(inplace=True)
df.isnull().sum()
df

,label,filtered,cleaned,content_tokens,normalisasi
0,1.0,mantap,mantap,['mantap'],['mantap']
1,-1.0,kenapa saya ditagih hutang kan saya di tolak p...,kenapa saya tagih hutang saya tolak pinjam man...,"['kenapa', 'saya', 'tagih', 'hutang', 'saya', ...","['kenapa', 'aku', 'tagih', 'hutang', 'aku', 't..."
2,1.0,sangat bagus,sangat bagus,"['sangat', 'bagus']","['sangat', 'bagus']"
3,1.0,bagus,bagus,['bagus'],['bagus']
4,1.0,ini pengajuan pertama moga di acc krn lg butuh...,aju pertama moga acc lg butuh dana kembang war...,"['aju', 'pertama', 'moga', 'acc', 'lg', 'butuh...","['aju', 'pertama', 'semoga', 'terima', 'lagi',..."
...,...,...,...,...,...
17636,-1.0,ada kami ngentod,ada kami ngentod,"['ada', 'kami', 'ngentod']","['ada', 'kami', 'ngentod']"
17637,-1.0,waktu jatuh tempo masih hari tapi panggilan ma...,jatuh tempo masih hari tapi panggil masuk tagi...,"['jatuh', 'tempo', 'masih', 'hari', 'tapi', 'p...","['jatuh', 'tempo', 'masih', 'hari', 'tapi', 'p..."
17638,-1.0,pelayanan dan csnya buruk blum waktunya byar s...,layan csnya buruk blum waktu byar sdah tagih k...,"['layan', 'csnya', 'buruk', 'blum', 'waktu', '...","['layan', 'csnya', 'buruk', 'blum', 'waktu', '..."
17639,-1.0,aplikasi busuk baru daftar udah di tolak penga...,aplikasi busuk baru daftar udah tolak aju pinj...,"['aplikasi', 'busuk', 'baru', 'daftar', 'udah'...","['aplikasi', 'busuk', 'baru', 'daftar', 'sudah..."


In [ ]:
# df = pd.DataFrame(df[['User','Score', 'Content']])
# df.tail(10)

In [ ]:
df['label'] = df['label'].replace([-1], '1') #Pos = 2
df['label'] = df['label'].replace([0], '0') #Neutral = 1
df['label'] = df['label'].replace([1], '2') #Negative = 0

In [ ]:
df['label'] = df['label'].replace(['negatif'], 1) #Pos = 2
df['label'] = df['label'].replace(['netral'], 2) #Neutral = 1
df['label'] = df['label'].replace(['positif'], 3) #Negative = 0

In [ ]:
df['label'].value_counts()


2    7753
1    7707
0    2056
Name: label, dtype: int64

In [ ]:
#split dataset kedalam data train dan test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.filtered, df.label,
                                                    test_size=0.2, random_state=15)


In [ ]:
#pembobotan kata using tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
x_train = vect.fit_transform(x_train)
x_train = x_train.toarray()
# np.array(x_train)
print(x_train)

x_test = vect.transform(x_test)
x_test = x_test.toarray()
# np.array(x_test)
print(x_test)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
#ubah data kedalam bentuk array
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_random_state
from sklearn.preprocessing import LabelEncoder


def projection_simplex(v, z=1):
    """
    Projection onto the simplex:
        w^* = argmin_w 0.5 ||w-v||^2 s.t. \sum_i w_i = z, w_i >= 0
    """
  
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w


class MulticlassSVM(BaseEstimator, ClassifierMixin):

    def __init__(self, C=1, max_iter=50, tol=0.05,
                 random_state=None, verbose=0):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol,
        self.random_state = random_state
        self.verbose = verbose

    def _partial_gradient(self, X, y, i):
        # Partial gradient for the ith sample.
        g = np.dot(X[i], self.coef_.T) + 1
        g[y[i]] -= 1
        return g

    def _violation(self, g, y, i):
        # Optimality violation for the ith sample.
        smallest = np.inf
        for k in range(g.shape[0]):
            if k == y[i] and self.dual_coef_[k, i] >= self.C:
                continue
            elif k != y[i] and self.dual_coef_[k, i] >= 0:
                continue

            smallest = min(smallest, g[k])

        return g.max() - smallest

    def _solve_subproblem(self, g, y, norms, i):
        # Prepare inputs to the projection.
        Ci = np.zeros(g.shape[0])
        Ci[y[i]] = self.C
        beta_hat = norms[i] * (Ci - self.dual_coef_[:, i]) + g / norms[i]
        z = self.C * norms[i]

        # Compute projection onto the simplex.
        beta = projection_simplex(beta_hat, z)

        return Ci - self.dual_coef_[:, i] - beta / norms[i]

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Normalize labels.
        self._label_encoder = LabelEncoder()
        y = self._label_encoder.fit_transform(y)

        # Initialize primal and dual coefficients.
        n_classes = len(self._label_encoder.classes_)
        self.dual_coef_ = np.zeros((n_classes, n_samples), dtype=np.float64)
        self.coef_ = np.zeros((n_classes, n_features))

        # Pre-compute norms.
        norms = np.sqrt(np.sum(X ** 2, axis=1))

        # Shuffle sample indices.
        rs = check_random_state(self.random_state)
        ind = np.arange(n_samples)
        rs.shuffle(ind)

        violation_init = None
        for it in range(self.max_iter):
            violation_sum = 0

            for ii in range(n_samples):
                i = ind[ii]

                # All-zero samples can be safely ignored.
                if norms[i] == 0:
                    continue

                g = self._partial_gradient(X, y, i)
                v = self._violation(g, y, i)
                violation_sum += v

                if v < 1e-12:
                    continue

                # Solve subproblem for the ith sample.
                delta = self._solve_subproblem(g, y, norms, i)

                # Update primal and dual coefficients.
                self.coef_ += (delta * X[i][:, np.newaxis]).T
                self.dual_coef_[:, i] += delta

            if it == 0:
                violation_init = violation_sum

            vratio = violation_sum / violation_init

            if self.verbose >= 1:
                print("iter", it + 1, "violation", vratio)

            if vratio < self.tol:
                if self.verbose >= 1:
                    print("Converged")
                break

        return self

    def predict(self, X):
        decision = np.dot(X, self.coef_.T)
        pred = decision.argmax(axis=1)
        return self._label_encoder.inverse_transform(pred)

In [ ]:
svm=MulticlassSVM(C=0.1, tol=0.01, max_iter=100, random_state=0, verbose=1)
svm.fit(x_train,y_train)

iter 1 violation 1.0
iter 2 violation 0.12252322898955126
iter 3 violation 0.039880892974538146
iter 4 violation 0.019012937143645943
iter 5 violation 0.010098957418336557
iter 6 violation 0.006464983931556973
Converged


MulticlassSVM(C=0.1, max_iter=100, random_state=0, tol=(0.01,), verbose=1)

In [ ]:
svm=MulticlassSVM(C=0.1, tol=0.01, max_iter=100, random_state=0, verbose=1)
svm.fit(x_train,y_train)

iter 1 violation 1.0
iter 2 violation 0.12252322898955126
iter 3 violation 0.039880892974538146
iter 4 violation 0.019012937143645943
iter 5 violation 0.010098957418336557
iter 6 violation 0.006464983931556973
Converged


MulticlassSVM(C=0.1, max_iter=100, random_state=0, tol=(0.01,), verbose=1)

In [ ]:
predict = svm.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("SVM Accuracy: ", accuracy_score(y_test,predict))
print("SVM Precision: ", precision_score(y_test,predict, average='macro'))
print("SVM Recall: ", recall_score(y_test,predict, average='macro'))
print("SVM f1_score: ", precision_score(y_test,predict, average='macro'))

print('====================================================\n')
print(classification_report(y_test, predict, zero_division=0))

SVM Accuracy:  0.8501712328767124
SVM Precision:  0.7843416716745795
SVM Recall:  0.7582954382173485
SVM f1_score:  0.7843416716745795

              precision    recall  f1-score   support

           0       0.60      0.48      0.53       403
           1       0.86      0.92      0.89      1585
           2       0.89      0.88      0.88      1516

    accuracy                           0.85      3504
   macro avg       0.78      0.76      0.77      3504
weighted avg       0.84      0.85      0.85      3504



In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# plt.figure(dpi=100)
# mat = confusion_matrix(y_test,predict, labels=[0,1,2])
# sns.heatmap(mat.T, annot=True, fmt='d', cbar=False,  xticklabels=['Negative',  'Neutral', 'Positive'], yticklabels=['Negative',  'Neutral', 'Positive'], )

# plt.title('Confusion Matrix')
# plt.xlabel('True label')
# plt.ylabel('Predicted label')
# plt.savefig("confusion_matrix.png")
# plt.show()

In [ ]:
report=classification_report(y_test,predict,labels=[1,2,3],target_names=['Negative',  'Neutral', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.79      0.99      0.87       762
     Neutral       1.00      0.01      0.01       147
    Positive       0.95      0.88      0.91       699

    accuracy                           0.85      1608
   macro avg       0.91      0.63      0.60      1608
weighted avg       0.88      0.85      0.81      1608



PREDIKSI DATASET BARU

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SKRIPSI/kredivoPrediksi.csv')
df.dropna(inplace=True)
df.isnull().sum()
df

,userName,score,at,content
0,Yolanna Pritiwi,5,2023-03-22 12:31:21,Membatu
1,AOMI gmz,5,2023-03-22 12:29:56,Sangat Terbantu
2,Lilis Siti Ambarwati,5,2023-03-22 12:28:16,Kredivo smart dan tercepat
3,Dika Restu Saputra,5,2023-03-22 12:27:07,Mantap sih apk ini bunga rendah
4,Zainal Abidin,5,2023-03-22 12:26:05,Mantap
...,...,...,...,...
9995,Nugroho Joko Lestariyo,5,2023-03-05 03:03:23,"Bagus untuk kebutuhan sehari-hari, tips bayar tepat waktu, pasti lancar"
9996,Mohamad Sodiq,5,2023-03-05 03:01:10,"aplikasi yang sangat bagus,, sangat membantu usaha saya"
9997,Abah Ompong,5,2023-03-05 02:56:57,Bagus
9998,Tyas Citra,5,2023-03-05 02:55:27,Paling oke


In [ ]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

df['content'] = df['content'].apply(lambda x: x.lower())
df['content'] = df['content'].apply(lambda x: remove_punctuations(x))

In [ ]:
df

,userName,score,at,content
0,Yolanna Pritiwi,5,2023-03-22 12:31:21,membatu
1,AOMI gmz,5,2023-03-22 12:29:56,sangat terbantu
2,Lilis Siti Ambarwati,5,2023-03-22 12:28:16,kredivo smart dan tercepat
3,Dika Restu Saputra,5,2023-03-22 12:27:07,mantap sih apk ini bunga rendah
4,Zainal Abidin,5,2023-03-22 12:26:05,mantap
...,...,...,...,...
9995,Nugroho Joko Lestariyo,5,2023-03-05 03:03:23,bagus untuk kebutuhan seharihari tips bayar tepat waktu pasti lancar
9996,Mohamad Sodiq,5,2023-03-05 03:01:10,aplikasi yang sangat bagus sangat membantu usaha saya
9997,Abah Ompong,5,2023-03-05 02:56:57,bagus
9998,Tyas Citra,5,2023-03-05 02:55:27,paling oke


In [ ]:
test = df['content']
time = df['at']
rating = df['score']

In [ ]:
#3
test = [str (item) for item in test]
test = vect.transform(test)
test = test.toarray()

In [ ]:
#4
pred_test = svm.predict(test)

In [ ]:
pred_test.shape

(10000,)

In [ ]:
sentiment = pd.Series(pred_test, name='sentiment')

In [ ]:
# menggabungkan dataframe dan series
df = pd.concat([df, sentiment], axis=1)

In [ ]:
df

,userName,score,at,content,sentiment
0,Yolanna Pritiwi,5,2023-03-22 12:31:21,membatu,2
1,AOMI gmz,5,2023-03-22 12:29:56,sangat terbantu,2
2,Lilis Siti Ambarwati,5,2023-03-22 12:28:16,kredivo smart dan tercepat,2
3,Dika Restu Saputra,5,2023-03-22 12:27:07,mantap sih apk ini bunga rendah,2
4,Zainal Abidin,5,2023-03-22 12:26:05,mantap,2
...,...,...,...,...,...
9995,Nugroho Joko Lestariyo,5,2023-03-05 03:03:23,bagus untuk kebutuhan seharihari tips bayar tepat waktu pasti lancar,2
9996,Mohamad Sodiq,5,2023-03-05 03:01:10,aplikasi yang sangat bagus sangat membantu usaha saya,2
9997,Abah Ompong,5,2023-03-05 02:56:57,bagus,2
9998,Tyas Citra,5,2023-03-05 02:55:27,paling oke,2


In [ ]:
df['sentiment'].value_counts()

2    6977
1    2573
0     450
Name: sentiment, dtype: int64

In [ ]:
df.to_csv("kredivoHasilSVM.csv", index = False)

In [ ]:
df_0 = df.loc[df['sentiment'] == 0]

In [ ]:
print(df_0)

Empty DataFrame
Columns: [userName, score, at, content, sentiment]
Index: []
